# checkpoint and reuse trained model with tf.train.Saver 's stored variables.

In [ ]:
# import libraries
import tensorflow as tf
import pandas as pd
import numpy as np
import sys
import datetime
import matplotlib.pyplot as plt
plt.style.use('ggplot') # use this plot style
%matplotlib inline

print('Python version ' + sys.version)
print('Tensorflow version ' + tf.VERSION)
print('Pandas version ' + pd.__version__)
print('Numpy version ' + np.__version__)

# Function to model
y = a * x^2 + b * x + c  

In [ ]:
## preparing the data

# Let's generate 1000 random samples
np.random.seed(41)
pool = np.random.rand(1000,1).astype(np.float32)

# Shuffle the samples
np.random.shuffle(pool)

# sample size of 15%
sample = int(1000*0.15)

# test and train data
test_x = pool[:sample]
train_x = pool[sample:]

print('Testing data points: ' + str(test_x.shape))
print('Training data points: ' + str(train_x.shape))

# Let's compute the ouput using 2 for a, 3 for b, and 5 for c
test_y = 2.0 * test_x**2 + 3.0 * test_x + 5
train_y = 2.0 * train_x**2 + 3.0 * train_x + 5

In [ ]:
df = pd.DataFrame({"x":train_x[:,0], "y":train_y[:,0]})
df.head()

In [ ]:
df.plot.scatter(x="x", y="y", figsize=(15,5))

# Model your Graph

Start to use W (for weight) and b (for bias) when setting up your variables. Aside from adding your ReLU activation function, it is a good idea to use Tensorflow's **matrix multiplication function (matmul)** as shown below.

> The ? in the shape output just means it can be of any shape.

For the shape parameter, you can think of it like this...
> shape = [how many data points do you have, how many features does each data point have]

For this lesson since we are doing a simple regression, we only have one feature (x). We use the **None** keyword so that we are not restricted on the number of samples to feed our model. This will become more important when you learn about training using batches on a future lesson.

In [ ]:
# you can adjust the number of neurons in the hidden layer here
hidden_size = 1

# placeholders
x = tf.placeholder(tf.float32, shape=[None, 1], name='x')
y = tf.placeholder(tf.float32, shape=[None, 1], name='y')

print("shape of x and y:")
print(x.get_shape(),y.get_shape())

In [ ]:
seed = 42

# create your first hidden layer!
# tf.truncated_normal([how many samples do you have, size of output layer])
W1 = tf.Variable(tf.truncated_normal([1, hidden_size], mean=0.1, stddev=0.01, seed=seed), name="w1")

# tf.truncated_normal([size of output layer])
b1 = tf.Variable(tf.zeros([hidden_size]), name="b1")

# shape of h1 = [size of your samples, size of output layer]
h1 = tf.nn.relu(tf.matmul(x, W1) + b1, name="h1")

print("shape of hidden layer:")
print(h1.get_shape())

In [ ]:
# Output Layer
W = tf.Variable(tf.truncated_normal([hidden_size, 1], mean=0.1, stddev=0.01, seed=seed), name="w")
b = tf.Variable(tf.zeros([1]), name="b")

# note that the input comes from our hidden layer h1
pred = tf.nn.relu(tf.matmul(h1, W) + b)

print("shape of output layer:")
print(pred.get_shape())

In [ ]:
loss = tf.reduce_mean(tf.square(pred-y))
optimizer = tf.train.GradientDescentOptimizer(0.09)
train = optimizer.minimize(loss)


# How Good is Your model?
Set up the following variables to calculate the accuracy rate of your model. You will do that shortly.

In [ ]:
correct_prediction = tf.equal(tf.round(pred), tf.round(y))
accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

# Training Time!
The best score I was able to obtain was a ~90% accuracy using a LR of 0.09 and iterating 250 times. I tried different learning rates and iterations but I was having a hard time getting past the 89 mark.

In [ ]:
init = tf.global_variables_initializer()
t=[]

with tf.Session() as sess:
    sess.run(init)
    #train the model
    train_data = {x:train_x, y:train_y}
    test_data = {x:test_x, y:test_y}
    print(W1.eval(), W.eval())
    for step in range(1000):
        train_loss, train_pred = sess.run([loss, train], feed_dict=train_data)
        if step%100==0:
            acc = sess.run(accuracy, feed_dict=train_data)
            t.append((step, train_loss))
            print("Training loss at step %d: %f  accuracy: %f" % (step, train_loss, acc))     
    
    print("Accuracy on the Training Set:", accuracy.eval(train_data) )
    print("Accuracy on the Test Set:", accuracy.eval(test_data) )
    print(W1.eval(), W.eval())
    
    #persist model into file
    save = tf.train.Saver()
    save_path = save.save(sess, 'checkpoint_models/lesson7.ckpt')
    print("Model saved in file: %s" % save_path)

In [ ]:
df_loss = pd.DataFrame(t, columns=['step', 'train_loss']) 
df_loss.plot(x='step', y='train_loss', figsize=(15,5));

## use checkpoint model to restore trained model

In [ ]:
rsaver = tf.train.Saver()
with tf.Session() as sess:
    rsaver.restore(sess, 'checkpoint_models/lesson7.ckpt')
    test_data = {x:test_x, y:test_y}
    print("Accuracy on the Test Set:", accuracy.eval(test_data) )